# Data preparation


In [ ]:
from IPython.display import Image
Image(filename='./WithoutAutoML.png',width=900)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import os
from azure.storage.blob import BlockBlobService

### Load energy demand data 

In [ ]:
demand = pd.read_csv('nyc_demand.csv', parse_dates=['timeStamp'])
weather = pd.read_csv('nyc_weather.csv', parse_dates=['timeStamp'])
demand = pd.merge(demand, weather, on=['timeStamp'], how='outer')
demand.head()

In [ ]:
plt_df = demand.loc[(demand.timeStamp>'2016-07-01') & (demand.timeStamp<='2016-07-07')]
plt.plot(plt_df['timeStamp'], plt_df['demand'])
plt.title('New York City power demand over one week in July 2017')
plt.xticks(rotation=45)
plt.show()

### Data Cleaning

Some periods in the time series are missing. This occurs if the period was missing in both the original demand and weather datasets. To identify these gaps, first we create an index of time periods that we would *expect* to be in the time series. There should be one record for every hour between the minimum and maximum datetimes in our dataset.

In [ ]:
min_time = min(demand['timeStamp'])
min_time

In [ ]:
max_time = max(demand['timeStamp'])
max_time

In [ ]:
dt_idx = pd.date_range(min_time, max_time, freq='H')
dt_idx

Now we index the dataframe according to this datetime index to insert missing records into the time series:

In [ ]:
demand.index = demand['timeStamp']
demand = demand.reindex(dt_idx)

Inserted missing records will have NaN/NaT values for all columns:

In [ ]:
demand[demand.isnull().all(axis=1)]

Now that there are no missing periods in the time series, we can start handling missing values by filling as many many as possible. Firstly, count the number of missing values in each column:

In [ ]:
demand.isnull().sum()

Missing timeStamp can be filled from the dataframe index:

In [ ]:
demand.loc[demand.isnull().all(axis=1), 'timeStamp'] = demand.loc[demand.isnull().all(axis=1)].index

For the other columns, we can fill many missing values by interpolating between the two closest non-missing values. Here, we use a quadratic function and set a limit of 6. This limit means that if more than 6 missing values occur consecutively, the missing values are not interpolated over and they remain missing. This is to avoid spurious interpolation between very distant time periods.

In [ ]:
demand = demand.interpolate(limit=6, method='linear')

Fill missing precip values with common value of 0:

In [ ]:
precip_mode = np.asscalar(stats.mode(demand['precip']).mode)
demand['precip'] = demand['precip'].fillna(precip_mode)

In [ ]:
demand.isnull().sum()

The number of missing values has now been greatly reduced. Records containing the remaining missing values will be removed later after model features have been created.

### Explore the data

By visualising the data, we can gain some intuition as to what kind of features could be helpful to the model.

In [ ]:
plt.hist(demand['demand'].dropna(), bins=100)
plt.title('Demand distribution')
plt.show()

In [ ]:
plt_df = demand.copy().loc[(demand['timeStamp']>='2016-01-01') & (demand['timeStamp']<'2017-01-01'), ]
plt.plot(plt_df['timeStamp'], plt_df['demand'], markersize=1)
plt.title('Hourly demand in 2016')
plt.ylabel('demand')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(demand['temp'].dropna(), bins=100)
plt.title('Temperature distribution')
plt.show()

In [ ]:
plt.plot(demand['temp'], demand['demand'], 'ro', markersize=1)
plt.title('Demand vs temperature')
plt.xlabel('temp')
plt.ylabel('demand')
plt.show()

The autocorrelation plot below shows the extent to which the demand variable correlates with itself at different intervals (lags). This plot shows that demand is highly autocorrelated over the closest 6 hour period.

In [ ]:
autocorrelation_plot(demand['demand'].dropna())
plt.xlim(0,24)
plt.title('Auto-correlation of hourly demand over a 24 hour period')
plt.show()

### Compute features for forecasting models

After data cleaning and missing values identification, data scientists need to do feature engineering.

Time Series data must be re-framed as a supervised learning dataset before we can start using machine learning algorithms.

We must choose the variable to be predicted and use feature engineering to construct all of the inputs that will be used to make predictions for future time steps.

-	Date Time Features: these are components of the time step itself for each observation.
-	Lag Features: these are values at prior time steps.
-	Window Features: these are a summary of values over a fixed window of prior time steps.

After exploring the data, it is clear that the energy demand follows seasonal trends, with daily, weekly and annual periodicity. We will create features that encode this information. First, we compute time driven features based on timeStamp. Note for dayofweek, Monday=0 and Sunday=6.

In [ ]:
demand_features = demand.copy()

In [ ]:
demand_features['hour'] = demand_features.timeStamp.dt.hour
demand_features['month'] = demand_features.timeStamp.dt.month-1
demand_features['dayofweek'] = demand_features.timeStamp.dt.dayofweek

Compute lagged demand features

In [ ]:
def generate_lagged_features(df, var, max_lag):
    for t in range(1, max_lag+1):
        df[var+'_lag'+str(t)] = df[var].shift(t, freq='1H')

In [ ]:
generate_lagged_features(demand_features, 'temp', 6)
generate_lagged_features(demand_features, 'demand', 6)

In [ ]:
demand_features.head()

### Final data cleaning

Count remaining null values.

In [ ]:
demand_features.isnull().sum()

Count number of rows with any null values

In [ ]:
demand_features.loc[demand_features.isnull().any(axis=1), ].shape[0]

This is a very small proportion of the overall dataset so can be safely dropped.

In [ ]:
demand_features.dropna(how='any', inplace=True)